In [2]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

df1_1 = pd.read_csv("survey1.1.csv", encoding='cp1252');
df1_2 = pd.read_csv("survey1.2.csv", encoding='cp1252');
df2_1 = pd.read_csv("survey2.1.csv", encoding='cp1252');
df2_2 = pd.read_csv("survey2.2.csv", encoding='cp1252');

In [3]:
dfmain = pd.concat([df1_1, df1_2, df2_1, df2_2])

In [4]:
toDrop = ['Date (Responses to recipient forms)', 'Recipient', 'Origin (Responses to recipient forms)', 'Primary key', 'Conditional: Portal Engagement Explanation', 'Consent', 'Flow', 'Display and Format', 'Content', 'Sex', 'Race:', 'Time Providing Care']

In [5]:
dropped_dfmain = dfmain.drop(columns = toDrop)
dropped_dfmain

,Score (Responses to recipient forms),Caregiver for Active Duty Military (Constituent Type),Caregiver for Veteran (Constituent Type),Other caregiver (Constituent Type),Veteran (Constituent Type),Other (Constituent Type),Age,Use of Video Conferencing,Depression,Motivation,Life Satisfaction,Physical Exhaustion,Desire To Escape,Same Self,Financial Decrease,Health Change,Demands Strength,Competing Demand,Concern About Future,Relationship Impact,Portal Experience,Portal Engagement,Portal Engagement Reinforcement,Portal Wellbeing Influence,Unhappy Alone,No One To Talk To,Cant Tolerate Being Alone,No One Understands Me,Waiting for Contact,Feel Completely Alone,Unable To Communicate,Need Company,Difficult to make friends,Feel Excluded,Hypertension (high blood pressure) (Chronic Conditions),Hyperlipidemia (high blood cholesterol or triglyceride levels) (Chronic Conditions),"Allergies, sinusitis and other upper respiratory conditions (Chronic Conditions)",Arthritis (Chronic Conditions),Mood Disorder (depression and bipolar disorder) (Chronic Conditions),Diabetes (Type1 and Type 2) (Chronic Conditions),Anxiety Disorder (Chronic Conditions),Asthma (Chronic Conditions),Coronary artery disease (includes myocardial infarction/heart attack) (Chronic Conditions),Thyroid disorder (Chronic Conditions),Chronic obstructive lung disease and bronchiectasis (Chronic Conditions),Traumatic Brain Injury (TBI) (Chronic Conditions),Post-Traumatic Stress Disorder (PTSD) (Chronic Conditions),None of the above (Chronic Conditions),Caregivers (Portal Video Use),Family members (Portal Video Use),Friends (Portal Video Use),Others (Portal Video Use),Not using Portal video (Portal Video Use),SMS/text (Connection Technology Use),Phone (Connection Technology Use),Video Portal (Connection Technology Use),Other (Connection Technology Use),Portal Chronic Disease Management Help
0,15,0,1,0,0,0,45 - 54,1+ Year,Yes,No,Neutral,Agree,Disagree,Agree,Strongly Agree,Strongly Agree,Strongly Agree,Neutral,Strongly Disagree,Agree,Strongly Agree,Strongly Agree,Strongly Agree,Agree,Rarely,Sometimes,Never,Never,Sometimes,Sometimes,Rarely,Sometimes,Sometimes,Sometimes,1,0,0,1,0,0,1,1,0,1,0,0,0,0,1,1,1,0,0,1,1,1,0,Agree
1,14,0,1,0,0,0,35 - 44,<3 Months,No,No,Strongly Disagree,Disagree,Strongly Disagree,Disagree,Disagree,Strongly Disagree,Strongly Disagree,Disagree,Strongly Disagree,Disagree,Agree,Agree,Agree,Neutral,Sometimes,Sometimes,Sometimes,Rarely,Never,Never,Never,Never,Rarely,Rarely,0,0,0,1,1,0,1,0,0,0,0,0,1,0,1,1,1,0,0,1,1,0,0,Agree
2,15,0,0,0,1,0,35 - 44,3 To 6 Months,No,No,Strongly Disagree,Strongly Disagree,Strongly Disagree,Neutral,Strongly Agree,Strongly Disagree,Strongly Disagree,Strongly Disagree,Strongly Disagree,Strongly Disagree,Neutral,Neutral,Agree,Agree,Never,Never,Rarely,Never,Never,Never,Never,Never,Never,Never,1,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,Agree
3,12,0,0,0,1,0,65+,6 To 9 Months,Yes,Yes,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,Agree,Agree,Agree,Agree,Sometimes,Sometimes,Sometimes,Sometimes,Sometimes,Sometimes,Sometimes,Sometimes,Sometimes,Sometimes,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,Agree
4,15,0,0,0,1,0,35 - 44,1+ Year,Yes,Yes,Strongly Disagree,Strongly Agree,Strongly Disagree,Neutral,Strongly Disagree,Strongly Disagree,Strongly Disagree,Strongly Disagree,Strongly Disagree,Neutral,Strongly Agree,Strongly Agree,Agree,Agree,Sometimes,Rarely,Sometimes,Sometimes,Rarely,Rarely,Rarely,Sometimes,Always,Rarely,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,1,1,0,0,0,1,1,0,Agree
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,12,0,0,0,1,0,45 - 54,6 To 9 Months,Yes,Yes,Agree,Agree,Disagree,Strongly Agree,Strongly Disagree,Agree,Strongly Agree,Agree,Strongly Disagree,Agree,Strongly Agree,Strongly Agree,Strongly Agree,Disagree,Sometimes,

In [6]:
dropped_dfmain.drop(list(dropped_dfmain.filter(regex = 'Constituent Type')), axis = 1, inplace = True)
#dropped_dfmain.drop(list(dropped_dfmain.filter(regex = 'Chronic Conditions')), axis = 1, inplace = True)
dropped_dfmain.drop(list(dropped_dfmain.filter(regex = 'Portal Video Use')), axis = 1, inplace = True)
dropped_dfmain.drop(list(dropped_dfmain.filter(regex = 'Connection Technology Use')), axis = 1, inplace = True)

In [11]:
dropped_dfmain['Use of Video Conferencing'].unique()


array(['1+ Year', '<3 Months', '3 To 6 Months', '6 To 9 Months',
       '9 To 12 Months'], dtype=object)

In [13]:
encoding = {
    'Agree': 1,
    'StronglyAgree': 2,
    'Strongly Agree': 2,
    'StronglyDisagree': -2,
    'Strongly Disagree': -2,
    'Disagree': -1,
    'Neutral': 0,
    'Yes': 1,
    'No': -1,
    'Rarely':-1,
    'Never':-2,
    'Sometimes':1,
    'Always':2,
    '<3 Months': -2,
    '3 To 6 Months': -1,
    '6 To 9 Months': 0,
    '9 To 12 Months': 1,
    '1+ Year': 2,
    '0 - 17': -2,
    '25-34': -1,
    '35 - 44': 0,
    '45 - 54': 1,
    '55 - 64': 2,
    '65+': 3
}
def f(s):
    l = []
    for b in s:
        #print(b)
        #print(type(b))
        if (b in encoding):
            l.append(encoding[b])
        elif isinstance(b, str) and b.isnumeric():
            l.append(int(b))
        else:
            l.append(b)
    return pd.Series(l)
    #print('\n')


In [14]:
data = dropped_dfmain.apply(f, axis = 0)
#data = pd.concat([data[data['Age'] == 2], data[data['Age'] == 3]]);
data = data.drop(columns = ['Age'])

In [9]:
data1 = data[data['Use of Video Conferencing'] == -2]
data1 = data1.drop(columns = ['Use of Video Conferencing'])
data2 = data[data['Use of Video Conferencing'] == -1]
data2 = data2.drop(columns = ['Use of Video Conferencing'])
data3 = data[data['Use of Video Conferencing'] == 1]
data3 = data3.drop(columns = ['Use of Video Conferencing'])
data4 = data[data['Use of Video Conferencing'] == 2]
data4 = data4.drop(columns = ['Use of Video Conferencing'])

In [10]:
print(data1.shape[0])
print(data2.shape[0])
print(data3.shape[0])
print(data4.shape[0])

70
32
39
75


In [11]:
pip install statsmodels 

Note: you may need to restart the kernel to use updated packages.


In [12]:
import statsmodels.stats.weightstats as ssw

results = []

for col in data1.columns: 
    vals1 = data1[col].to_list()
    vals2 = data4[col].to_list()
    
    # Null hypothesis: means are the same
    # Alternative: the mean after is higher
    
    cm = ssw.CompareMeans(ssw.DescrStatsW(vals1), ssw.DescrStatsW(vals2))
    res = cm.ztest_ind(alternative='smaller',usevar='unequal')
    msg = "Cannot reject the null"
    if (res[1] <= 0.05):
        msg = "Alternative accepted: mean is larger"
    results.append([col, res[1], msg])

res = pd.DataFrame(results, columns=['Item', 'P-Value', 'Verdict'])
res

,Item,P-Value,Verdict
0,Score (Responses to recipient forms),0.089370,Cannot reject the null
1,Depression,0.740689,Cannot reject the null
2,Motivation,0.856135,Cannot reject the null
3,Life Satisfaction,0.404179,Cannot reject the null
4,Physical Exhaustion,0.781424,Cannot reject the null
5,Desire To Escape,0.581741,Cannot reject the null
6,Same Self,0.984413,Cannot reject the null
7,Financial Decrease,0.928777,Cannot reject the null
8,Health Change,0.903245,Cannot reject the null
9,Demands Strength,0.587247,Cannot reject the null


In [13]:
results = []

for col in data1.columns: 
    vals1 = data1[col].to_list()
    vals2 = data4[col].to_list()
    
    # Null hypothesis: means are the same
    # Alternative: the mean after is higher
    
    cm = ssw.CompareMeans(ssw.DescrStatsW(vals1), ssw.DescrStatsW(vals2))
    res = cm.ztest_ind(alternative='larger',usevar='unequal')
    msg = "Cannot reject the null"
    if (res[1] <= 0.05):
        msg = "Alternative accepted: mean is smaller"
    results.append([col, res[1], msg])

res = pd.DataFrame(results, columns=['Item', 'P-Value', 'Verdict'])
res

,Item,P-Value,Verdict
0,Score (Responses to recipient forms),0.910630,Cannot reject the null
1,Depression,0.259311,Cannot reject the null
2,Motivation,0.143865,Cannot reject the null
3,Life Satisfaction,0.595821,Cannot reject the null
4,Physical Exhaustion,0.218576,Cannot reject the null
5,Desire To Escape,0.418259,Cannot reject the null
6,Same Self,0.015587,Alternative accepted: mean is smaller
7,Financial Decrease,0.071223,Cannot reject the null
8,Health Change,0.096755,Cannot reject the null
9,Demands Strength,0.412753,Cannot reject the null


In [14]:
mdata1 = pd.concat([data1, data2])
mdata2 = pd.concat([data3, data4])

In [15]:
results = []

for col in data1.columns: 
    vals1 = mdata1[col].to_list()
    vals2 = mdata2[col].to_list()
    
    # Null hypothesis: means are the same
    # Alternative: the mean after is higher
    
    cm = ssw.CompareMeans(ssw.DescrStatsW(vals1), ssw.DescrStatsW(vals2))
    res = cm.ztest_ind(alternative='smaller',usevar='unequal')
    msg = "Cannot reject the null"
    if (res[1] <= 0.05):
        msg = "Alternative accepted: mean is larger"
    results.append([col, res[1], msg])

res = pd.DataFrame(results, columns=['Item', 'P-Value', 'Verdict'])
res

,Item,P-Value,Verdict
0,Score (Responses to recipient forms),0.184054,Cannot reject the null
1,Depression,0.249852,Cannot reject the null
2,Motivation,0.706437,Cannot reject the null
3,Life Satisfaction,0.317440,Cannot reject the null
4,Physical Exhaustion,0.397462,Cannot reject the null
5,Desire To Escape,0.202235,Cannot reject the null
6,Same Self,0.915287,Cannot reject the null
7,Financial Decrease,0.891595,Cannot reject the null
8,Health Change,0.838642,Cannot reject the null
9,Demands Strength,0.433930,Cannot reject the null


In [16]:
results = []

for col in data1.columns: 
    vals1 = mdata1[col].to_list()
    vals2 = mdata2[col].to_list()
    
    # Null hypothesis: means are the same
    # Alternative: the mean after is higher
    
    cm = ssw.CompareMeans(ssw.DescrStatsW(vals1), ssw.DescrStatsW(vals2))
    res = cm.ztest_ind(alternative='larger',usevar='unequal')
    msg = "Cannot reject the null"
    if (res[1] <= 0.05):
        msg = "Alternative accepted: mean is smaller"
    results.append([col, res[1], msg])

res = pd.DataFrame(results, columns=['Item', 'P-Value', 'Verdict'])
res

,Item,P-Value,Verdict
0,Score (Responses to recipient forms),0.815946,Cannot reject the null
1,Depression,0.750148,Cannot reject the null
2,Motivation,0.293563,Cannot reject the null
3,Life Satisfaction,0.682560,Cannot reject the null
4,Physical Exhaustion,0.602538,Cannot reject the null
5,Desire To Escape,0.797765,Cannot reject the null
6,Same Self,0.084713,Cannot reject the null
7,Financial Decrease,0.108405,Cannot reject the null
8,Health Change,0.161358,Cannot reject the null
9,Demands Strength,0.566070,Cannot reject the null
